In [2]:
#Import Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Conv1D, MaxPooling1D, Dropout, Flatten, RepeatVector, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
import tensorflow_addons as tfa

2023-12-18 09:59:17.155238: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-18 09:59:20.444308: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 09:59:20.444398: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 09:59:21.147476: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-18 09:59:22.756909: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-18 09:59:22.769269: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
# File paths
train_file_1 = '/home/disi/ff/Datasets/Dataset_2/Train1.csv'
train_file_2 = '/home/disi/ff/Datasets/Dataset_2/Train2.csv'
test_file = '/home/disi/ff/Datasets/Dataset_2/Test.csv'

# Load the datasets
train_data_1 = pd.read_csv(train_file_1)
train_data_2 = pd.read_csv(train_file_2)
test_data = pd.read_csv(test_file)

In [4]:
# Combine training data and sort all data by 'created_at' if it exists
train_data = pd.concat([train_data_1, train_data_2], ignore_index=True)
if 'Date' in train_data.columns:
    train_data['Date'] = pd.to_datetime(train_data['Date'])
    train_data = train_data.sort_values(by='Date')
if 'Date' in test_data.columns:
    test_data['Date'] = pd.to_datetime(test_data['Date'])
    test_data = test_data.sort_values(by='Date')

# Selecting the relevant columns
forecast_cols = ['Temperature', 'Humidity', 'Light']
train_forecast = train_data[forecast_cols]
test_forecast = test_data[forecast_cols]

# Scale the data
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_forecast)
test_scaled = scaler.transform(test_forecast)

In [6]:
# Function to create sequences
def create_sequences(data, input_seq_length, output_seq_length):
    X, y = [], []
    for i in range(len(data) - input_seq_length - output_seq_length + 1):
        seq_x = data[i:i + input_seq_length]
        seq_y = data[i + input_seq_length:i + input_seq_length + output_seq_length]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Sequence parameters
input_seq_length = 10
output_seq_length = 3
features = 3  # Number of features

# Create sequences for training and testing
X_train, y_train = create_sequences(train_scaled, input_seq_length, output_seq_length)
X_test, y_test = create_sequences(test_scaled, input_seq_length, output_seq_length)

# Clear Keras session
K.clear_session()


input_layer = Input(shape=(input_seq_length, features))

In [7]:
# Convolutional layers with BatchNormalization and regularization
x = Conv1D(16, (2), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(input_layer)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=(2))(x)
x = Conv1D(32, (2), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(x)
x = BatchNormalization()(x)
x = Conv1D(64, (2), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
x = RepeatVector(output_seq_length)(x)

# LSTM layers with dropout
x = LSTM(100, activation='relu', dropout=0.2, return_sequences=True, kernel_regularizer=regularizers.l2(0.001))(x)
x = LSTM(100, activation='relu', dropout=0.2, return_sequences=True, kernel_regularizer=regularizers.l2(0.001))(x)
output = TimeDistributed(Dense(3, activation = 'sigmoid'))(x)

model = Model(inputs=input_layer, outputs=output)

# Optimizer with custom learning rate
#optimizer = Adam(learning_rate=0.0001, weight_decay=1e-5)
optimizer = tfa.optimizers.AdamW(learning_rate=0.0001, weight_decay=0.000001)

# Compile model with regularization loss
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])

# Early stopping and learning rate reduction on plateau
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)

model.summary()

2023-12-18 10:01:08.884700: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10, 3)]           0         
                                                                 
 conv1d (Conv1D)             (None, 10, 16)            112       
                                                                 
 batch_normalization (Batch  (None, 10, 16)            64        
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 5, 16)             0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 5, 32)             1056      
                                                                 
 batch_normalization_1 (Bat  (None, 5, 32)             128   

In [8]:
# Implement early stopping
#early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), verbose=1) #callbacks=[early_stopping])


Epoch 1/50
169/169 [==============================] - 6s 15ms/step - loss: 0.5320 - accuracy: 0.8158 - val_loss: 0.4511 - val_accuracy: 0.6170
Epoch 2/50
169/169 [==============================] - 2s 12ms/step - loss: 0.3214 - accuracy: 0.8806 - val_loss: 0.2811 - val_accuracy: 0.9151
Epoch 3/50
169/169 [==============================] - 2s 12ms/step - loss: 0.2068 - accuracy: 0.8841 - val_loss: 0.1735 - val_accuracy: 0.9605
Epoch 4/50
169/169 [==============================] - 2s 12ms/step - loss: 0.1358 - accuracy: 0.8884 - val_loss: 0.1121 - val_accuracy: 0.9508
Epoch 5/50
169/169 [==============================] - 2s 12ms/step - loss: 0.0919 - accuracy: 0.8871 - val_loss: 0.0764 - val_accuracy: 0.9679
Epoch 6/50
169/169 [==============================] - 2s 11ms/step - loss: 0.0649 - accuracy: 0.8931 - val_loss: 0.0545 - val_accuracy: 0.9787
Epoch 7/50
169/169 [==============================] - 2s 11ms/step - loss: 0.0480 - accuracy: 0.8940 - val_loss: 0.0409 - val_accuracy: 0.9803

In [9]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, precision_score, recall_score, f1_score
import numpy as np

# Make predictions
y_pred = model.predict(X_test)

# Flatten the predictions and true values for simplicity
y_true_flat = y_test.flatten()
y_pred_flat = y_pred.flatten()

# Calculate MAE, RMSE, and MAPE
mae = mean_absolute_error(y_true_flat, y_pred_flat)
rmse = np.sqrt(mean_squared_error(y_true_flat, y_pred_flat))
mape = np.mean(np.abs((y_true_flat - y_pred_flat) / y_true_flat)) * 100

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"MAPE: {mape}%")

305/305 [==============================] - 1s 3ms/step
MAE: 0.024107178130097953
RMSE: 0.03567554811447036
MAPE: inf%


/tmp/ipykernel_18668/521214265.py:14: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_true_flat - y_pred_flat) / y_true_flat)) * 100
